In [1]:
from pathlib import Path
import altair as alt
import duckdb
from utils import (
    loads_cae_db,
    executes_query_duckdb
)

In [2]:
cae_path = Path(
    "../../kids_chances/data/raw/cae_history.txt"
)

destionation_db = Path("../data/cae_db.duckdb")
cae = loads_cae_db(cae_path, destionation_db)

In [3]:
query = """
select distinct(tipo_beneficiario) from cae limit 10000"""
executes_query_duckdb(query, cae)

,tipo_beneficiario
0,EGRESO
1,SIN CREDITO POR NO MATRICULA O POR SUSPENSION
2,NUEVO BENEFICIARIO
3,GRATUIDAD
4,BENEFICIARIO RENOVANTE
5,DESERCION


In [4]:
query_sum = """

SELECT id_2023 as id,
año_licitacion as año_pedido,
        genero_registrado as genero,
        dependencia_colegio as colegio,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        COUNT(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' THEN tipo_beneficiario END)
            as total_prestamos,
        COUNT(DISTINCT(nombre_carrera || '-' || nombre_ies)) as cantidad_carreras,
        COUNT(CASE WHEN arancel_solicitado > 0 then arancel_solicitado END)
            as años_financiados,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'EGRESO' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_egresos,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'DESERCION' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_deserciones,
        ROUND(SUM(total_prestado_ajustado),0) as total_prestado

FROM cae
GROUP BY id, genero, colegio, region, quintil, año_pedido
"""

In [7]:
final_table_to_query = executes_query_duckdb(query_sum, cae)
final_table_to_query.to_parquet("../data/summary_table.parquet")

In [6]:
final_table_to_query

,id,año_pedido,genero,colegio,region,quintil,total_prestamos,cantidad_carreras,años_financiados,total_egresos,total_deserciones,total_prestado
0,473.995,2011,F,Particular subvencionado,REGION DE VALPARAISO,quintil 1 y 2,1,1,3,1,0,5860950.0
1,508.761,2008,F,Estudiante de Colegio o Liceo Particular subve...,REGION DE LA ARAUCANIA,quintil 1 y 2,1,1,1,0,1,3063020.0
2,459.170,2014,M,Municipal,REGION DE ÑUBLE,quintil 1 y 2,1,2,4,1,0,7045383.0
3,455.088,2011,M,Municipal,REGION METROPOLITANA,quintil 1 y 2,1,1,1,0,1,2922826.0
4,468.460,2011,M,Particular subvencionado,REGION DEL BIO-BIO,quintil 1 y 2,1,1,6,1,0,14811895.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1290988,1.218.978,2024,M,Particular Subvencionado,REGION DEL BIO-BIO,quintil 4,1,1,0,0,0,0.0
1290989,272.790,2010,F,Municipal,REGION DEL MAULE,quintil 1 y 2,1,1,1,0,0,814450.0
1290990,400.575,2009,F,Municipal,REGION DEL LIB. B. O'HIGGINS,quintil 1 y 2,1,1,1,0,0,1397916.0
1290991,187.784,2007,F,Particular subvencionado,REGION DEL MAULE,quintil 1 y 2,1,1,1,0,0,732478.0
